##Demonstration code of Graph Search to search solution of 8 Puzzle Game
###Dr. Trung T. Nguyen

This Python notebook demonstrates how to setup the 8 Puzzle game as a graph search to find its solution using A*. There are also additional code to visualize the steps to move from the given initial state to the goal state using text drawing.

In [1]:
# define the EightPuzzleNode class
# the state of an 8-puzzle game board is 3x3 cells => flattend to 1D of a list of 9 elements
# so first three elements belongs to first row, ...
class EightPuzzleNode:
    def __init__(self, state, parent, move, depth, cost, key):
        self.state = state    # current state
        self.parent = parent  # the parent node
        self.move = move      # the move to reach this state UP, DOWN, LEFT, RIGHT
        self.depth = depth    # the depth value
        self.cost = cost      # path cost so far
        self.key = key        # the f=g+h value used in A*
        if self.state:
            self.map = ''.join(str(e) for e in self.state)  # the string concatenation of all cells, easily to compare
    def __eq__(self, other):
        return self.map == other.map
    def __lt__(self, other):
        return self.map < other.map

In [2]:
# define EightPuzzleProblem class with embedding search algorithms
from collections import deque
from heapq import heappush, heappop, heapify
import itertools

class EightPuzzleProblem:
    def __init__(self, initial):
        """Initialize internal variables"""
        self.initial_state = initial                      # the initial state of the game
        self.goal_state = [0, 1, 2, 3, 4, 5, 6, 7, 8]     # the goal state of the game
        self.goal_node = None                             # game node default as None
        self.board_len = 9                                # length of the game board - 9 cells
        self.board_side = 3                               # length of each side - 3 x 3

    def move(self, state, direction):
        """Create new states by trying moving in four directions"""
        new_state = state[:]
        index = new_state.index(0) # find the index of the blank cell
        if direction == 1:  # Up
            if index >= self.board_side:
                temp = new_state[index - self.board_side]
                new_state[index - self.board_side] = new_state[index]
                new_state[index] = temp
                return new_state
            else:
                return None
        if direction == 2:  # Down
            if index < self.board_len - self.board_side:
                temp = new_state[index + self.board_side]
                new_state[index + self.board_side] = new_state[index]
                new_state[index] = temp
                return new_state
            else:
                return None
        if direction == 3:  # Left
            if index % self.board_side != 0:
                temp = new_state[index - 1]
                new_state[index - 1] = new_state[index]
                new_state[index] = temp
                return new_state
            else:
                return None
        if direction == 4:  # Right
            if index % self.board_side != self.board_side - 1:
                temp = new_state[index + 1]
                new_state[index + 1] = new_state[index]
                new_state[index] = temp
                return new_state
            else:
                return None

    def expand(self, node):
        """Expand the given node to new new neighbor nodes by trying all possible moves"""
        neighbors = list()
        # Test UP
        neighbors.append(EightPuzzleNode(self.move(node.state, 1), node, 1, node.depth + 1, node.cost + 1, 0))
        # Test DOWN
        neighbors.append(EightPuzzleNode(self.move(node.state, 2), node, 2, node.depth + 1, node.cost + 1, 0))
        # Test Left
        neighbors.append(EightPuzzleNode(self.move(node.state, 3), node, 3, node.depth + 1, node.cost + 1, 0))
        # Test RIGHT
        neighbors.append(EightPuzzleNode(self.move(node.state, 4), node, 4, node.depth + 1, node.cost + 1, 0))
        # get the list of possible neighbor nodes after trying to move in fourth direction
        nodes = [neighbor for neighbor in neighbors if neighbor.state]
        return nodes

    def backtrace(self):
        """Backtrace the path from goal node to initial node to get the list of moves"""
        moves = list()
        current_node = self.goal_node
        while self.initial_state != current_node.state:
            if current_node.move == 1:
                movement = 'Up'
            elif current_node.move == 2:
                movement = 'Down'
            elif current_node.move == 3:
                movement = 'Left'
            else:
                movement = 'Right'
            moves.insert(0, movement)
            current_node = current_node.parent
        return moves

    def print_board(self, state):
        """Print the game board from the given state"""
        print('+---+---+---+')
        print('|',state[0],'|',state[1],'|',state[2],'|')
        print('+---+---+---+')
        print('|',state[3],'|',state[4],'|',state[5],'|')
        print('+---+---+---+')
        print('|',state[6],'|',state[7],'|',state[8],'|')
        print('+---+---+---+')

    def h(self, state):
        """The heuristic function which measure how close the current state to the goal state by distance"""
        return sum(abs(b % self.board_side - g % self.board_side) + abs(b//self.board_side - g//self.board_side)
               for b, g in ((state.index(i), self.goal_state.index(i)) for i in range(1, self.board_len)))

    def astar(self, start_state):
        """ Perform A star search from the given start state.
            This version uses the heap data structure in Python to automatically sort the node by the f=g+h values
        """
        explored, heap, heap_entry, counter = set(), list(), {}, itertools.count()
        key = self.h(start_state)
        root = EightPuzzleNode(start_state, None, None, 0, 0, key)
        entry = (key, 0, root)
        heappush(heap, entry)
        heap_entry[root.map] = entry
        while heap:
            entry = heappop(heap)
            explored.add(entry[2].map)
            if entry[2].state == self.goal_state: # found solution
                self.goal_node = entry[2]
                return heap
            neighbors = self.expand(entry[2])
            for neighbor in neighbors:
                neighbor.key = neighbor.cost + self.h(neighbor.state) #estimate f(n) = g(n) + h(n)
                entry = (neighbor.key, neighbor.move, neighbor)
                if neighbor.map not in explored: # make sure not repeat visited state
                    heappush(heap, entry)
                    explored.add(neighbor.map)
                    heap_entry[neighbor.map] = entry
                elif neighbor.map in heap_entry and neighbor.key < heap_entry[neighbor.map][2].key:
                    hindex = heap.index((heap_entry[neighbor.map][2].key,
                                         heap_entry[neighbor.map][2].move,
                                         heap_entry[neighbor.map][2]))
                    heap[int(hindex)] = entry
                    heap_entry[neighbor.map] = entry
                    heapify(heap)

In [4]:
# setup the initial state and search problem
initial_state = [3,2,5,4,1,8,6,0,7]
#initial_state = [0,2,1,3,5,4,6,7,8]
#initial_state = [3,4,5,6,7,8,0,1,2]
#initial_state = [3,4,5,0,1,2,6,7,8]
problem = EightPuzzleProblem(initial_state)

# perform search
frontier = problem.astar(initial_state)

# check solution
if frontier is None:
    print('No solution found!!!')
else:
    moves = problem.backtrace()
    print("path_to_goal: " + str(moves))
    print("cost_of_path: " + str(len(moves)))
    ######
    move_map = { 'Up': 1, 'Down': 2, 'Left': 3, 'Right': 4 }
    problem.print_board(problem.initial_state)
    cur_state = problem.initial_state
    for move in moves:
        cur_state = problem.move(cur_state, move_map[move])
        print('Next move: ', move)
        problem.print_board(cur_state)

path_to_goal: ['Right', 'Up', 'Up', 'Left', 'Down', 'Left', 'Up']
cost_of_path: 7
+---+---+---+
| 3 | 2 | 5 |
+---+---+---+
| 4 | 1 | 8 |
+---+---+---+
| 6 | 0 | 7 |
+---+---+---+
Next move:  Right
+---+---+---+
| 3 | 2 | 5 |
+---+---+---+
| 4 | 1 | 8 |
+---+---+---+
| 6 | 7 | 0 |
+---+---+---+
Next move:  Up
+---+---+---+
| 3 | 2 | 5 |
+---+---+---+
| 4 | 1 | 0 |
+---+---+---+
| 6 | 7 | 8 |
+---+---+---+
Next move:  Up
+---+---+---+
| 3 | 2 | 0 |
+---+---+---+
| 4 | 1 | 5 |
+---+---+---+
| 6 | 7 | 8 |
+---+---+---+
Next move:  Left
+---+---+---+
| 3 | 0 | 2 |
+---+---+---+
| 4 | 1 | 5 |
+---+---+---+
| 6 | 7 | 8 |
+---+---+---+
Next move:  Down
+---+---+---+
| 3 | 1 | 2 |
+---+---+---+
| 4 | 0 | 5 |
+---+---+---+
| 6 | 7 | 8 |
+---+---+---+
Next move:  Left
+---+---+---+
| 3 | 1 | 2 |
+---+---+---+
| 0 | 4 | 5 |
+---+---+---+
| 6 | 7 | 8 |
+---+---+---+
Next move:  Up
+---+---+---+
| 0 | 1 | 2 |
+---+---+---+
| 3 | 4 | 5 |
+---+---+---+
| 6 | 7 | 8 |
+---+---+---+
